# Home Assignment

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_feather('/Users/idodavid/Downloads/home_assignment.feather')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 881 entries, HKTLMYY to TLJYWBE
dtypes: float64(794), int64(43), object(44)
memory usage: 4.8+ GB


In [3]:
import cufflinks as cf; cf.go_offline()

## Data Integrity

### Duplicated rows

In [4]:
df.duplicated().sum()

0

### Data types

In [5]:
columns_type = df.dtypes

In [6]:
columns_type.value_counts()

float64    794
object      44
int64       43
dtype: int64

### Fixing value issues

In [7]:
df.replace('nan',np.nan, inplace=True)
df.replace('-∞', float('-inf'), inplace=True)
df.replace('False', False, inplace=True)
df.replace('FALSE', False, inplace=True)
df.replace('True', True, inplace=True)
df.replace('TRUE', True, inplace=True)

In [8]:
cat_columns = df.dtypes[df.dtypes == 'object'].index
for c in cat_columns:
    if c in ['TQSLCKT',
             'FRPZZAO',
             'CJICMVV',
             'CNECOOI',
             'HLUJJDC',
             'OYBONYX',
             'DRELDXX',
             'IMAZODZ',
             'ABMWXIO',
             'EYAUITY',
             'TOTBAZC',
             'TERIJOV',
             'FKCIOEN']:
        continue
    df[c]=df[c].astype(float)

In [9]:
df.dtypes.value_counts()

float64    825
int64       43
object      13
dtype: int64

## Handling duplicated columns

In [10]:
def nan_positions_match(df, col1, col2):
    return (df[col1].isnull() == df[col2].isnull()).all()

def categorical_distribution_match(df, col1, col2):
    # Get value counts for each column
    vc1 = df[col1].value_counts()
    vc2 = df[col2].value_counts()
    is_equal_distribution = vc1.values.tolist() == vc2.values.tolist()
    category_matching = list(zip(vc1.index, vc2.index))

    # Check if indices for each category match
    all_categories_match = True
    for cat_1, cat_2 in category_matching:
        indices_col1 = df[df[col1] == cat_1].index
        indices_col2 = df[df[col2] == cat_2].index
        if not indices_col1.equals(indices_col2):
            all_categories_match = False
            break

    # Return results
    if is_equal_distribution and all_categories_match:
        return True
    else:
        return False

def check_columns_identity(df, col1, col2):
    if df[col1].dtype != df[col2].dtype:
        return False
    if not nan_positions_match(df, col1, col2):
        return False
    if df[col1].dtype == 'object':
        if not categorical_distribution_match(df, col1, col2):
            return False
    elif df[col1].dtype == 'int64':
        if not categorical_distribution_match(df, col1, col2):
            return False
    elif df[col1].dtype == 'float64':
        if not (df[col1].dropna().rank() == df[col2].dropna().rank()).all():
            return False
    return True

In [11]:
cols = df.columns

In [12]:
from collections import defaultdict
cols_to_drop = set()
matches = defaultdict(list)
sample = df.sample(1000, random_state=42)
for i,col1 in enumerate(cols):
    print(f'Testing column {i} of {len(cols)}. Number of cols to drop so far: {len(cols_to_drop)}', end='\r')
    for col2 in cols:
        if col1 == col2:
            continue
        if check_columns_identity(sample, col1, col2):
            #print(f'Fount match: {col1} to {col2}')
            matches[col1].append(col2)
            cols_to_drop.add(col2)

In [13]:
true_dup = defaultdict(list)
cols_to_drop = set()
for col, duplicates in matches.items():
    if col in cols_to_drop:
        continue
    for d in duplicates:
        if d in cols_to_drop:
            continue
        if check_columns_identity(df, col, d):
            print(f'Indeed, column {d} is a duplicate of column {col}')
            true_dup[col].append(d)
            cols_to_drop.add(d)
        else:
            print(f'Eventually, column {d} is not a duplicate of column {col}')

Eventually, column XKLWARB is not a duplicate of column MSVWNFK
Eventually, column UIDFQDA is not a duplicate of column MSVWNFK
Eventually, column MSVWNFK is not a duplicate of column XKLWARB
Eventually, column UIDFQDA is not a duplicate of column XKLWARB
Eventually, column MSVWNFK is not a duplicate of column UIDFQDA
Eventually, column XKLWARB is not a duplicate of column UIDFQDA
Indeed, column MMGUYQW is a duplicate of column HGCJPXQ
Indeed, column EJLLSVL is a duplicate of column HGCJPXQ
Indeed, column FGRQLRJ is a duplicate of column HGCJPXQ
Eventually, column LWFSOBB is not a duplicate of column HGCJPXQ
Eventually, column KHDGXRJ is not a duplicate of column HGCJPXQ
Eventually, column MZUTRGY is not a duplicate of column HGCJPXQ
Eventually, column WSTYRNH is not a duplicate of column HGCJPXQ
Eventually, column IJOSAYZ is not a duplicate of column HGCJPXQ
Eventually, column GNJJUIH is not a duplicate of column HGCJPXQ
Eventually, column MKVXAUF is not a duplicate of column HGCJPXQ


In [14]:
for col,duplicates in true_dup.items():
    print(col, duplicates)

HGCJPXQ ['MMGUYQW', 'EJLLSVL', 'FGRQLRJ']
TQSLCKT ['FRPZZAO']
CJICMVV ['CNECOOI', 'HLUJJDC']
KHDGXRJ ['MZUTRGY', 'VXZZORS', 'KXGNEZR']
WSTYRNH ['IJOSAYZ', 'HOAJWUR']
GNJJUIH ['MKVXAUF', 'NJFNKOX']
OYBONYX ['DRELDXX', 'IMAZODZ', 'ABMWXIO', 'EYAUITY', 'TOTBAZC']
WYYWAWQ ['FMWCHDN', 'ERXNSUN', 'UANVOYU', 'MKGQLST', 'BADTSBX', 'NEKQCXA', 'YXRYJNT', 'TWTFEPM', 'ZBSCEOT', 'FBTIPCA', 'YLYAAGH', 'CYGFXWC', 'AHMUWMD']
WPJXHBR ['PGENXWU']
KAHHGFH ['MOYGMAB']
KVOYKGE ['RNCEKOH', 'UMIGGHZ', 'ZHPRSBN']
TERIJOV ['FKCIOEN']
SWMNPFK ['BXBUVRU', 'KRMCXMU', 'XRUSUCA', 'YPMGCVT', 'HTMOHRO', 'WWHSAAX', 'LVDPQQT', 'EHAPOTE', 'PINNYVT', 'LNGBURD', 'BQEDTSY', 'LGRGANO', 'IPLONQP', 'RQVRYXC', 'WMZGUWK', 'VWPFOKV', 'AKRJZVC', 'VDOFXVB', 'TMSJMYI', 'ELXUMTI', 'KNYSSTU']
OJSWUAF ['EBEXNTO', 'FYWFJXP', 'JEQWBOK', 'TGXFVBT', 'WCYWJXK']
XXIPGDP ['XJTQVGI', 'VMYGDBS']
JZVNCXT ['SHLZRCB', 'XIVOAXI', 'KKMOSNN', 'WTARSLQ', 'OPLXQJS']
JMFGDPB ['UILLRWJ', 'CKTRRMV', 'KYJTYYF']
SETVSTN ['RTYTHAC', 'MXNLYUW', 'QDZIZBN']
DU

### Removing unneeded columns

In [16]:
for col, duplicates in true_dup.items():
    df.drop(columns=duplicates, inplace=True)    

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 770 entries, HKTLMYY to TLJYWBE
dtypes: float64(744), int64(22), object(4)
memory usage: 4.2+ GB


In [18]:
nuniques = df.nunique(dropna=False)
df.drop(columns=nuniques[nuniques == 1].index, inplace=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726288 entries, 0 to 726287
Columns: 767 entries, HKTLMYY to TLJYWBE
dtypes: float64(742), int64(21), object(4)
memory usage: 4.2+ GB


In [20]:
df.to_feather('removed_duplicated_columns.feather')